In [2]:
## Use the pseudo label and the mixup technique to push the trendency into the training set
import json
import os
import numpy as np
import pandas as pd
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from catboost import Pool, cv, CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import * 
from sklearn.metrics import *
import gc

In [90]:
all_data = pd.read_csv('../data/feature_data_530.csv',low_memory=False)
# all_data = pd.read_csv('../data/feature_data_530.csv')
# glove
glove_tags = pd.read_csv('../data/alltags_feature.csv')
glove_title = pd.read_csv('../data/title_feature.csv')
all_data = pd.concat([all_data, glove_tags, glove_title], axis=1)

In [91]:
pseudo_label = json.load(open('../data/best7225.json', encoding="utf-8"))
test_resuts = [sample['popularity_score'] for sample in pseudo_label['result']]
test_results_df = pd.DataFrame(test_resuts)
test_results_df.columns = ['label']

In [92]:
del glove_tags
del glove_title

In [93]:
train_all_data = all_data[all_data['train_type'] != -1]
submit_all_data = all_data[all_data['train_type'] == -1]
del all_data

train_all_data = train_all_data.reset_index(drop=True)
submit_all_data = submit_all_data.reset_index(drop=True)

submit_all_data['label'] = test_results_df['label']
# train_all_data = train_all_data.append(submit_all_data, ignore_index=True)
# train_all_data = train_all_data.reset_index(drop=True)
# del submit_all_data

In [94]:
submit_all_data

,Pid,train_type,Uid,Uid_count,mean_label,Category,Subcategory,Concept,Title_len,Title_number,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,50731,-1,48249,1,0.000000,6,13,352,8,2,...,-0.121735,0.021623,0.204132,-0.358730,0.233779,-0.401930,0.174435,-0.555150,0.088762,-0.334895
1,96454,-1,58858,11,8.705455,3,71,573,231,44,...,0.024520,-0.129912,-0.017224,-0.007224,0.062414,0.021764,-0.037089,-0.096281,0.130909,0.014050
2,172136,-1,58858,11,8.705455,2,33,253,54,9,...,-0.146950,-0.262795,0.107282,0.182885,0.082495,0.202486,0.063760,-0.098770,0.110895,0.053098
3,177707,-1,27151,176,4.247443,3,71,232,36,6,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
4,177708,-1,27151,176,4.247443,3,71,232,36,6,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180576,1118324,-1,46088,2,4.380000,0,35,301,6,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
180577,1118329,-1,12280,7,1.382857,0,60,110,25,4,...,0.134555,0.186670,0.330580,0.227405,0.155600,-0.088244,0.144885,-0.199950,0.033038,-0.065841
180578,1118336,-1,43496,9,3.674444,4,28,186,41,7,...,0.156818,-0.107772,0.157098,-0.079490,0.042594,-0.034590,-0.349488,0.016242,-0.172004,-0.012735
180579,1118349,-1,5492,17,5.671176,7,27,54,15,3,...,0.142362,-0.097873,-0.026855,0.195713,-0.101540,0.240027,-0.119417,-0.012023,0.021850,-0.081305


In [95]:
feature_columns = ['Pid', 'train_type', 'mean_label'] 
feature_columns += ['user_fe_{}'.format(i) for i in range(399)]
feature_columns += ['loc_fe_{}'.format(i) for i in range(400)]
submit_all_data = submit_all_data.drop(feature_columns, axis=1)

In [96]:
submit_all_data 

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,48249,1,6,13,352,8,2,96,15,500,...,-0.121735,0.021623,0.204132,-0.358730,0.233779,-0.401930,0.174435,-0.555150,0.088762,-0.334895
1,58858,11,3,71,573,231,44,324,35,375,...,0.024520,-0.129912,-0.017224,-0.007224,0.062414,0.021764,-0.037089,-0.096281,0.130909,0.014050
2,58858,11,2,33,253,54,9,126,15,375,...,-0.146950,-0.262795,0.107282,0.182885,0.082495,0.202486,0.063760,-0.098770,0.110895,0.053098
3,27151,176,3,71,232,36,6,141,13,500,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
4,27151,176,3,71,232,36,6,141,13,500,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180576,46088,2,0,35,301,6,1,78,13,500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
180577,12280,7,0,60,110,25,4,164,23,500,...,0.134555,0.186670,0.330580,0.227405,0.155600,-0.088244,0.144885,-0.199950,0.033038,-0.065841
180578,43496,9,4,28,186,41,7,85,9,500,...,0.156818,-0.107772,0.157098,-0.079490,0.042594,-0.034590,-0.349488,0.016242,-0.172004,-0.012735
180579,5492,17,7,27,54,15,3,305,35,500,...,0.142362,-0.097873,-0.026855,0.195713,-0.101540,0.240027,-0.119417,-0.012023,0.021850,-0.081305


In [18]:
## generate the samples from the testset, the ratio of the mixup could be 0.7 or 0.5, let's set it as 0.
submit_all_data.columns[:-600]

Index(['Uid', 'Uid_count', 'Category', 'Subcategory', 'Concept', 'Title_len',
       'Title_number', 'Alltags_len', 'Alltags_number', 'img_length',
       'img_width', 'pixel', 'img_model', 'svd_mode_t_0', 'svd_mode_t_1',
       'svd_mode_t_2', 'svd_mode_t_3', 'svd_mode_t_4', 'svd_mode_t_5',
       'svd_mode_t_6', 'svd_mode_t_7', 'svd_mode_t_8', 'svd_mode_t_9',
       'svd_mode_t_10', 'svd_mode_t_11', 'svd_mode_t_12', 'svd_mode_t_13',
       'svd_mode_t_14', 'svd_mode_t_15', 'svd_mode_t_16', 'svd_mode_t_17',
       'svd_mode_t_18', 'svd_mode_t_19', 'svd_mode_0', 'svd_mode_1',
       'svd_mode_2', 'svd_mode_3', 'svd_mode_4', 'svd_mode_5', 'svd_mode_6',
       'svd_mode_7', 'svd_mode_8', 'svd_mode_9', 'svd_mode_10', 'svd_mode_11',
       'svd_mode_12', 'svd_mode_13', 'svd_mode_14', 'svd_mode_15',
       'svd_mode_16', 'svd_mode_17', 'svd_mode_18', 'svd_mode_19', 'Mediatype',
       'hour', 'day', 'weekday', 'week_hour', 'year_weekday', 'Longitude',
       'Latitude', 'Geoaccuracy', 'phot

In [82]:
submit_all_data.iloc[0,2]

6

In [86]:
data_aug = pd.DataFrame(np.random.random((180580,684)),columns=submit_all_data.columns)
# data_aug.columns = submit_all_data.columns
data_aug.iloc[0,0] = submit_all_data.iloc[0,0]
data_aug.iloc[0,1] = submit_all_data.iloc[0,1] + 1
data_aug.iloc[0,2:4] = submit_all_data.iloc[0,2:4]
data_aug.iloc[0,5:8] = submit_all_data.iloc[0,5:8] * 0.7 + submit_all_data.iloc[1,5:8] * 0.3

In [ ]:
## column index removed: +1
## column index keep the same: 0,2-4,9-12,53-58,61,63-69,81
## column index plus: 13-52, -600:,59,60,62,70-80,82,5-8,

In [97]:
from tqdm import trange
data_aug = pd.DataFrame(np.random.random((180580,683)),columns=submit_all_data.columns)
# aug_index = 0
for index in trange(len(submit_all_data)-1):
#     data_aug.iloc[0,0] = submit_all_data.iloc[0,0]
# data_aug.iloc[0,1] = submit_all_data.iloc[0,1] + 1
# data_aug.iloc[0,2:4] = submit_all_data.iloc[0,2:4]
# data_aug.iloc[0,5:8] = submit_all_data.iloc[0,5:8] * 0.7 + submit_all_data.iloc[1,5:8] * 0.3
    data_aug.iloc[index,0] = submit_all_data.iloc[index,0]
    data_aug.iloc[index,1] = submit_all_data.iloc[index,1] + 1
    data_aug.iloc[index,2:4] = submit_all_data.iloc[index,2:4]
    data_aug.iloc[index,5:8] = submit_all_data.iloc[index,5:8] * 0.7 + submit_all_data.iloc[index + 1,5:8] * 0.3
    data_aug.iloc[index,9:12] = submit_all_data.iloc[index,9:12]
    data_aug.iloc[index,13:52] = submit_all_data.iloc[index,13:52] * 0.7 + submit_all_data.iloc[index + 1,13:52] * 0.3
    data_aug.iloc[index,53:58] = submit_all_data.iloc[index,53:58]
    data_aug.iloc[index,59:60] = submit_all_data.iloc[index,59:60] * 0.7 + submit_all_data.iloc[index + 1,59:60] * 0.3
    data_aug.iloc[index,61] = submit_all_data.iloc[index,61]
    data_aug.iloc[index,62] = submit_all_data.iloc[index,62] * 0.7 + submit_all_data.iloc[index + 1,62] * 0.3
    data_aug.iloc[index,63:69] = submit_all_data.iloc[index,63:69]
    data_aug.iloc[index,70:80] = submit_all_data.iloc[index,70:80] * 0.7 + submit_all_data.iloc[index + 1,70:80] * 0.3
    data_aug.iloc[index,81] = submit_all_data.iloc[index,81]
    data_aug.iloc[index,82] = submit_all_data.iloc[index,82] * 0.7 + submit_all_data.iloc[index + 1,82] * 0.3
    data_aug.iloc[index,-600:] = submit_all_data.iloc[index,-600:] * 0.7 + submit_all_data.iloc[index + 1,-600:] * 0.3
    # data_aug.loc[index][13:52] = submit_all_data.loc[index][13:52] * 0.7 + submit_all_data.loc[index + 1][13:52] * 0.3
    # data_aug.loc[index][5:8] = submit_all_data.loc[index][5:8] * 0.7 + submit_all_data.loc[index + 1][5:8] * 0.3
    # data_aug.loc[index][59:60] = submit_all_data.loc[index][59:60] * 0.7 + submit_all_data.loc[index + 1][59:60] * 0.3
    # data_aug.loc[index][62] = submit_all_data.loc[index][62] * 0.7 + submit_all_data.loc[index + 1][62] * 0.3
    # data_aug.loc[index][70:80] = submit_all_data.loc[index][70:80] * 0.7 + submit_all_data.loc[index + 1][70:80] * 0.3
    # data_aug.loc[index][82] = submit_all_data.loc[index][82] * 0.7 + submit_all_data.loc[index + 1][82] * 0.3
    # data_aug.loc[index][-600:] = submit_all_data.loc[index][-600:] * 0.7 + submit_all_data.loc[index + 1][-600:] * 0.3
    
# data_aug.columns = submit_all_data.columns
    
    
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180580/180580 [29:10<00:00, 103.14it/s]


In [98]:
data_aug

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,48249.0,2.0,6.0,13.0,0.711738,74.9,14.6,164.4,0.063644,500.0,...,-0.077858,-0.023838,0.137725,-0.253278,0.182369,-0.274822,0.110978,-0.417489,0.101406,-0.230211
1,58858.0,12.0,3.0,71.0,0.018565,177.9,33.5,264.6,0.689355,375.0,...,-0.026921,-0.169777,0.020128,0.049809,0.068439,0.075981,-0.006834,-0.097028,0.124905,0.025765
2,58858.0,12.0,2.0,33.0,0.837066,48.6,8.1,130.5,0.619430,375.0,...,-0.046023,-0.242478,-0.045467,0.123014,0.153426,0.236537,0.118119,-0.050547,0.102166,-0.015352
3,27151.0,177.0,3.0,71.0,0.065149,36.0,6.0,141.0,0.042160,500.0,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
4,27151.0,177.0,3.0,71.0,0.164143,36.0,6.0,141.0,0.196153,500.0,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180575,43016.0,19.0,4.0,12.0,0.524489,10.2,1.7,135.4,0.790257,500.0,...,0.076573,0.176876,0.297346,-0.080689,0.187132,-0.152289,-0.032265,-0.127572,0.095165,0.180264
180576,46088.0,3.0,0.0,35.0,0.531895,11.7,1.9,103.8,0.759124,500.0,...,0.040366,0.056001,0.099174,0.068221,0.046680,-0.026473,0.043466,-0.059985,0.009911,-0.019752
180577,12280.0,8.0,0.0,60.0,0.952215,29.8,4.9,140.3,0.370405,500.0,...,0.141234,0.098337,0.278535,0.135336,0.121698,-0.072148,-0.003427,-0.135092,-0.028475,-0.049909
180578,43496.0,10.0,4.0,28.0,0.537602,33.2,5.8,151.0,0.060552,500.0,...,0.152481,-0.104802,0.101912,0.003071,-0.000646,0.047795,-0.280467,0.007762,-0.113848,-0.033306


In [99]:
data_aug['Uid'] = pd.DataFrame(data_aug['Uid'], dtype='int')
data_aug['Uid_count'] = pd.DataFrame(data_aug['Uid_count'], dtype='int')
data_aug['Category'] = pd.DataFrame(data_aug['Category'], dtype='int')
data_aug['Subcategory'] = pd.DataFrame(data_aug['Subcategory'], dtype='int')
data_aug['Concept'] = pd.DataFrame(data_aug['Concept'], dtype='int')
data_aug['img_length'] = pd.DataFrame(data_aug['img_length'], dtype='int')
data_aug['img_width'] = pd.DataFrame(data_aug['img_width'], dtype='int')
data_aug['pixel'] = pd.DataFrame(data_aug['pixel'], dtype='int')
data_aug['img_model'] = pd.DataFrame(data_aug['img_model'], dtype='int')
# 'hour', 'day', 'weekday', 'week_hour', 'year_weekday'
data_aug['Mediatype'] = pd.DataFrame(data_aug['Mediatype'], dtype='int')
data_aug['hour'] = pd.DataFrame(data_aug['hour'], dtype='int')
data_aug['day'] = pd.DataFrame(data_aug['day'], dtype='int')
data_aug['weekday'] = pd.DataFrame(data_aug['weekday'], dtype='int')
data_aug['week_hour']] = pd.DataFrame(data_aug['day'], dtype='int')
data_aug['year_weekday'] = pd.DataFrame(data_aug['weekday'], dtype='int')
# 'Geoaccuracy', 'photo_count', 'ispro', 'firstdate',
#        'firstweek', 'firstmonth', 'firstdatetaken', 'firstdatetakenweek',
#        'firstdatetakenmonth'
data_aug['Geoaccuracy'] = pd.DataFrame(data_aug['Geoaccuracy'], dtype='int')
data_aug['ispro'] = pd.DataFrame(data_aug['ispro'], dtype='int')
data_aug['firstdate'] = pd.DataFrame(data_aug['firstdate'], dtype='int')
data_aug['firstweek'] = pd.DataFrame(data_aug['firstweek'], dtype='int')

data_aug['firstmonth'] = pd.DataFrame(data_aug['firstmonth'], dtype='int')
data_aug['firstdatetaken'] = pd.DataFrame(data_aug['firstdatetaken'], dtype='int')
data_aug['firstdatetakenweek'] = pd.DataFrame(data_aug['firstdatetakenweek'], dtype='int')
data_aug['firstdatetakenmonth'] = pd.DataFrame(data_aug['firstdatetakenmonth'], dtype='int')
# 'Ispublic'
data_aug['Ispublic'] = pd.DataFrame(data_aug['Ispublic'], dtype='int')






In [100]:
data_aug

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,48249,2,6,13,0,74.9,14.6,164.4,0.063644,500,...,-0.077858,-0.023838,0.137725,-0.253278,0.182369,-0.274822,0.110978,-0.417489,0.101406,-0.230211
1,58858,12,3,71,0,177.9,33.5,264.6,0.689355,375,...,-0.026921,-0.169777,0.020128,0.049809,0.068439,0.075981,-0.006834,-0.097028,0.124905,0.025765
2,58858,12,2,33,0,48.6,8.1,130.5,0.619430,375,...,-0.046023,-0.242478,-0.045467,0.123014,0.153426,0.236537,0.118119,-0.050547,0.102166,-0.015352
3,27151,177,3,71,0,36.0,6.0,141.0,0.042160,500,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
4,27151,177,3,71,0,36.0,6.0,141.0,0.196153,500,...,0.189475,-0.195072,-0.401882,-0.016685,0.318932,0.315990,0.244958,0.061973,0.081797,-0.175070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180575,43016,19,4,12,0,10.2,1.7,135.4,0.790257,500,...,0.076573,0.176876,0.297346,-0.080689,0.187132,-0.152289,-0.032265,-0.127572,0.095165,0.180264
180576,46088,3,0,35,0,11.7,1.9,103.8,0.759124,500,...,0.040366,0.056001,0.099174,0.068221,0.046680,-0.026473,0.043466,-0.059985,0.009911,-0.019752
180577,12280,8,0,60,0,29.8,4.9,140.3,0.370405,500,...,0.141234,0.098337,0.278535,0.135336,0.121698,-0.072148,-0.003427,-0.135092,-0.028475,-0.049909
180578,43496,10,4,28,0,33.2,5.8,151.0,0.060552,500,...,0.152481,-0.104802,0.101912,0.003071,-0.000646,0.047795,-0.280467,0.007762,-0.113848,-0.033306


In [101]:
data_aug = data_aug.fillna('0')
data_aug.to_csv('../data/test_data_aug.csv', header=True, index=None)